In [ ]:


from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import numpy as np
import pandas as pd
from torch import tensor

In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/Dataset/adult_train.csv")
test_df = pd.read_csv("/content/drive/MyDrive/Dataset/adult_test.csv")
train_df.head()

,age,fnlwgt,educational-num,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income,log_capital_gain,log_capital_loss,Private_Workclass,education_label,marital_label,white_collar_label,gender_male,native_country_label,high_income
0,81,36147,15,0,1,1,10605,0,2,38,50,9.269175,0.0,1,0,1,0,1,1,0
1,46,525848,7,0,1,1,0,0,48,38,50,0.000000,0.0,0,0,1,0,1,1,1
2,39,77516,13,1,1,1,2174,0,40,38,50,7.684784,0.0,0,1,0,1,1,1,1
3,39,179137,9,1,1,0,0,0,40,38,50,0.000000,0.0,1,0,0,1,0,1,1
4,46,215943,9,4,1,0,0,0,40,38,50,0.000000,0.0,1,0,0,1,0,1,1


In [ ]:
train_feature_df = train_df[train_df.columns[:-1]].astype(float)
train_features = tensor(train_feature_df.values, dtype=torch.float)
train_labels = tensor(train_df.high_income, dtype=torch.int)

max_value, _ = train_features.max(dim=0)
train_features /= max_value

In [ ]:
test_feature_df = test_df[test_df.columns[:-1]].astype(float)
test_features = tensor(test_feature_df.values, dtype=torch.float)
test_labels = tensor(test_df.high_income, dtype=torch.int)

max_value, _ = test_features.max(dim=0)
test_features /= max_value

# Linear Model to Neural Net

In [ ]:
def init_weights():
  torch.manual_seed(42)
  n_params = train_features.shape[1]
  weights = torch.rand(n_params, 1) # now it is a 1-D matrix
  return weights.requires_grad_()

In [ ]:
x = tensor([1,2,3]) # example
x

tensor([1, 2, 3])

In [ ]:
x[:,None]

tensor([[1],
        [2],
        [3]])

In [ ]:
train_labels = train_labels[:, None]
test_labels = test_labels[:, None]

In [ ]:
train_labels.shape

torch.Size([38388, 1])

In [ ]:
def calculate_prediction(weights, features):
  multi = features @ weights # @ also means matrix multiplication
  preds = torch.sigmoid(multi)
  return preds

In [ ]:
def calculate_loss(preds, labels):
  return torch.abs(preds-labels).mean()

In [ ]:
def update_weights(weights, lr):
  weights.sub_(weights.grad * lr)
  return

In [ ]:
def one_epoch(epoch_no, weights, lr):
  preds = calculate_prediction(weights, train_features)
  loss = calculate_loss(preds, train_labels)
  loss.backward()
  with torch.no_grad(): update_weights(weights, lr)
  print(f"Epoch {epoch_no+1} => Loss: {loss}")

In [ ]:
def train_model(epochs, lr):
  weights = init_weights()
  for epoch_no in range(epochs):
    one_epoch(epoch_no, weights, lr)
  return weights

In [ ]:
weights = train_model(epochs=15, lr=2)

Epoch 1 => Loss: 0.24599476158618927
Epoch 2 => Loss: 0.24555248022079468
Epoch 3 => Loss: 0.2447742372751236
Epoch 4 => Loss: 0.24382245540618896
Epoch 5 => Loss: 0.24285228550434113
Epoch 6 => Loss: 0.2419712096452713
Epoch 7 => Loss: 0.24123184382915497
Epoch 8 => Loss: 0.2406444549560547
Epoch 9 => Loss: 0.24019527435302734
Epoch 10 => Loss: 0.23986051976680756
Epoch 11 => Loss: 0.239615336060524
Epoch 12 => Loss: 0.2394377738237381
Epoch 13 => Loss: 0.23930998146533966
Epoch 14 => Loss: 0.2392183542251587
Epoch 15 => Loss: 0.239152729511261


In [ ]:
def accuracy(weights, features):
  preds = calculate_prediction(weights, features)
  results = test_labels == (preds > 0.5).int()
  return results.float().mean()

In [ ]:
accuracy(weights, test_features)

tensor(0.7611)

# Shallow Neural Network

In [ ]:
torch.manual_seed(42)

def init_weights(n_hidden=10):
  n_params = train_features.shape[1]
  hidden_layer = torch.rand(n_params, n_hidden)
  head_layer = torch.rand(n_hidden, 1)
  bias = torch.rand(1)[0]
  return hidden_layer.requires_grad_(), head_layer.requires_grad_(), bias.requires_grad_()

In [ ]:
def calculate_prediction(weights, features):
    hidden_layer, head_layer, bias = weights
    hidden_layer_out = torch.sigmoid(features @ hidden_layer)
    head_layer_out = hidden_layer_out @ head_layer + bias
    return torch.sigmoid(head_layer_out)

In [ ]:
def update_weights(weights, lr):
  for layer in weights:
    layer.sub_(layer.grad * lr)
  return

In [ ]:
weights = train_model(epochs=15, lr=2)

Epoch 1 => Loss: 0.24125881493091583
Epoch 2 => Loss: 0.241155207157135
Epoch 3 => Loss: 0.24096600711345673
Epoch 4 => Loss: 0.24072103202342987
Epoch 5 => Loss: 0.24045276641845703
Epoch 6 => Loss: 0.24018868803977966
Epoch 7 => Loss: 0.2399476021528244
Epoch 8 => Loss: 0.239739328622818
Epoch 9 => Loss: 0.2395666241645813
Epoch 10 => Loss: 0.23942774534225464
Epoch 11 => Loss: 0.23931866884231567
Epoch 12 => Loss: 0.23923443257808685
Epoch 13 => Loss: 0.23917017877101898
Epoch 14 => Loss: 0.23912166059017181
Epoch 15 => Loss: 0.23908522725105286


In [ ]:
torch.save(weights, "shallow_nnet_weights.pt")

In [ ]:
accuracy(weights, test_features)

tensor(0.7611)